## Redes neuronales recurrentes y otros modelos de secuencia

Se crea una instancia de una red neuronal recurrente (RNN). Aquí, torch.nn.RNN es la clase que implementa una RNN básica en PyTorch. Los argumentos 300 y 512 especifican los tamaños de las características de entrada y de las características ocultas, respectivamente.

- 300: Este es el tamaño de la característica de cada elemento de entrada en la secuencia. Por ejemplo, si estás procesando datos de texto, esto podría ser el tamaño del vector de características (como embeddings de palabras) para cada palabra.
- 512: Este es el tamaño de la característica de los estados ocultos en la RNN. Esto afecta la capacidad del modelo para aprender dependencias complejas en los datos. Un número más alto puede permitir que el modelo capture dependencias más complejas, pero también puede llevar a un mayor riesgo de sobreajuste y requerir más memoria y tiempo de cómputo.

Esta línea de código prepara un modelo RNN que puedes entrenar con datos adecuados para realizar tareas como la predicción de series temporales, el modelado de lenguaje, y otras tareas donde los datos de entrada tienen una naturaleza secuencial.

In [ ]:
import torch

model = torch.nn.RNN(300, 512)
     

#### RNN usando Pytorch

El código siguiente define y utiliza una celda de red neuronal recurrente (RNN) personalizada usando PyTorch y la biblioteca fastai.

In [ ]:
import fastai
# fastai.__version__
     
from fastai.text.all import *
     

class RNNCell(nn.Module):    

    def __init__(self, input_size, hidden_size):
        super(RNNCell, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.ih = nn.Linear(input_size, hidden_size)
        self.hh = nn.Linear(hidden_size, hidden_size)

    def forward(self, x, h = None):
        if h is None:
            h = torch.zeros(x.size(0), self.hidden_size)
        h = torch.tanh(self.ih(x) + self.hh(h))
        return h
     

#oculto
cell = RNNCell(100, 300)
cell(torch.randn(1, 100)).shape


En el código anterior, la clase RNNCell, define una celda RNN personalizada. La celda tiene dos partes principales:

- Inicialización (__init__) que define dos capas lineales, una para transformar la entrada (ih) y otra para transformar el estado oculto anterior (hh).
- Paso hacia adelante (forward) que calcula el nuevo estado oculto combinando la entrada actual x y el estado oculto anterior h (si h no se proporciona, se inicializa como un tensor de ceros). La suma de las transformaciones lineales de x y h se pasa a través de una función de activación tanh para obtener el nuevo estado oculto.

Luego se crea una instancia de RNNCell con un tamaño de entrada de 100 y un tamaño de estado oculto de 300. Luego, pasa un tensor aleatorio (simulando una entrada única de tamaño 100) a través de la celda RNN para obtener el estado oculto, e imprime la forma del tensor resultante, que sería [1, 300], correspondiente a un lote de tamaño 1 con un vector de estado oculto de tamaño 300.

Ahora definimos una clase RNN que implementa una red neuronal recurrente completa usando la clase RNNCell personalizada anteriormente:

In [ ]:
class RNN(nn.Module):    

    def __init__(self, input_size, hidden_size):
        super(RNN, self).__init__()
        self.cell = RNNCell(input_size, hidden_size)

    def forward(self, x, h = None):
        
        print(x.shape)
        for i in range(x.shape[1]):
            h = self.cell(x[:,i], h)
            
        return h
     

#oculto
rnn = RNN(100, 300)
rnn(torch.randn(256, 10, 100)).shape
     


Se define la clase RNN como una subclase de nn.Module, que es la clase base para todos los módulos de red neuronal en PyTorch. En el constructor (__init__), inicializa una instancia de RNNCell con los tamaños de entrada y ocultos especificados, almacenándola como self.cell.

El método forward define cómo los datos pasan a través de la red:

- x.shape imprime la forma de la entrada x. Suponiendo que x tiene forma (256, 10, 100), esto indica que estamos procesando 256 secuencias, cada una de longitud 10, donde cada elemento de la secuencia tiene 100 características.

- El bucle `for i in range(x.shape[1])` itera sobre la segunda dimensión de x, que es la longitud de la secuencia (10 en este caso). En cada iteración, extrae el i-ésimo elemento de cada secuencia (usando x[:, i]), y lo pasa a self.cell junto con el estado oculto actual h.

- h se actualiza en cada paso de tiempo al procesar el elemento correspondiente de la secuencia a través de la celda RNN. Si h es None inicialmente, será inicializado a un tensor de ceros dentro de RNNCell.


Luego se crea una instancia de RNN con un tamaño de entrada de 100 y un tamaño de estado oculto de 300. Luego, se pasa un tensor de entrada aleatorio con forma `(256, 10, 100)` a través de la RNN. Este tensor representa 256 secuencias, cada una de longitud 10, con cada elemento de la secuencia teniendo 100 características.

El resultado de `rnn(torch.randn(256, 10, 100))`.shape sería la forma del último estado oculto producido, que es [256, 300]. Esto indica que hay un estado oculto de tamaño 300 para cada una de las 256 secuencias después de procesar todos los elementos de cada secuencia.

### Ejemplo de clasificación de texto

Definimos una clase TextClassifier que es un modelo de red neuronal para clasificación de texto, utilizando módulos comunes de PyTorch.

In [ ]:
class TextClassifier(nn.Module):
    
    def __init__(self, vocab_size, hidden_size):
        super(TextClassifier, self).__init__()
        self.emb = nn.Embedding(vocab_size, hidden_size)
        self.rnn = RNN(hidden_size, hidden_size)
        self.fc1 = nn.Linear(hidden_size, 10)
        self.fc2 = nn.Linear(10, 1)
        
    def forward(self, x):
        
        x = self.emb(x)
        x = self.rnn(x)
        x = self.fc1(x)
        out = self.fc2(x)
        
        return out
     

path = untar_data(URLs.IMDB)
dls = TextDataLoaders.from_folder(path, valid='test', bs=256)
     

dls.show_batch(max_n=5)     

Explicación del código:

In [ ]:
class TextClassifier(nn.Module):
    def __init__(self, vocab_size, hidden_size):
        super(TextClassifier, self).__init__()
        self.emb = nn.Embedding(vocab_size, hidden_size)
        self.rnn = RNN(hidden_size, hidden_size)
        self.fc1 = nn.Linear(hidden_size, 10)
        self.fc2 = nn.Linear(10, 1)


- nn.Embedding(vocab_size, hidden_size): Este módulo es una tabla de búsqueda que almacena embeddings de un vocabulario fijo de tamaño vocab_size, cada uno con una dimensión hidden_size. Transforma índices de palabras en vectores densos.

- RNN(hidden_size, hidden_size): Se utiliza una RNN, definida previamente, que toma la salida del embedding como entrada. La RNN puede capturar dependencias secuenciales en el texto.

- nn.Linear(hidden_size, 10) y nn.Linear(10, 1): Estas son capas lineales que transforman la salida de la RNN a dimensiones más pequeñas, eventualmente produciendo una salida única. Esto es típico para tareas de clasificación binaria donde la salida puede representar la probabilidad de una clase (a menudo después de aplicar una función sigmoide).

In [ ]:
def forward(self, x):
    x = self.emb(x)        # Aplica embedding al texto de entrada
    x = self.rnn(x)        # Procesa el texto a través de la RNN
    x = self.fc1(x)        # Primera capa lineal
    out = self.fc2(x)      # Segunda capa lineal que produce la salida final
    return out


El método forward define cómo los datos pasan a través del modelo. Los datos de entrada (x) son índices de palabras que primero se convierten en embeddings, luego se procesan a través de una RNN, y finalmente se transforman a través de dos capas lineales para producir la salida.

In [ ]:
path = untar_data(URLs.IMDB)
dls = TextDataLoaders.from_folder(path, valid='test', bs=256)
dls.show_batch(max_n=5)


- untar_data(URLs.IMDB): Descarga y descomprime el conjunto de datos IMDB para análisis de sentimientos, que contiene críticas de películas etiquetadas como positivas o negativas.
- TextDataLoaders.from_folder(path, valid='test', bs=256): Crea cargadores de datos para el entrenamiento y validación. Los datos se dividen en un conjunto de entrenamiento y un conjunto de validación, con un tamaño de lote de 256.
- dls.show_batch(max_n=5): Muestra un lote de datos de ejemplo, que puede ser útil para verificar cómo se están cargando y procesando los datos.

**Ejercicio 1:** Modifica la arquitectura del TextClassifier para incluir una capa adicional de LSTM en lugar de RNN simple. Investiga y compara los resultados en términos de precisión y pérdida durante el entrenamiento.

Pasos:
- Define una nueva clase LSTMCell o modifica la clase RNN para utilizar nn.LSTM en lugar de nn.RNN.
- Integra esta nueva celda LSTM en el TextClassifier.
- Entrena el modelo modificado en el conjunto de datos IMDB y compara los resultados con la versión original.

**Ejercicio 2:** Evalúa cómo el tamaño del vocabulario afecta la precisión de la clasificación.

Pasos:
- Experimenta con diferentes tamaños de vocabulario (por ejemplo, 5000, 10000, 20000) al crear los TextDataLoaders.
- Entrena tu modelo TextClassifier para cada tamaño de vocabulario.
- Compara los resultados en términos de precisión y rendimiento computacional.

**Ejercicio 3:** Visualiza los embeddings de palabras utilizando técnicas como t-SNE o PCA para entender cómo las palabras relacionadas están agrupadas en el espacio vectorial.

Pasos:

- Entrena el modelo TextClassifier y extrae la matriz de embeddings.
- Utiliza t-SNE o PCA (disponible en sklearn) para reducir la dimensionalidad de los embeddings a 2 o 3 dimensiones.
- Visualiza los embeddings reducidos usando Matplotlib o Seaborn y analiza si palabras semánticamente similares se agrupan juntas.

**Ejercicio 4:** Implementa técnicas de regularización como Dropout o L2 regularization para mejorar la generalización del modelo.

Pasos:
- Modifica la clase TextClassifier para incluir capas de Dropout después de cada capa RNN y antes de las capas lineales.
- Entrena el modelo y compara el rendimiento en el conjunto de validación para evaluar si la regularización reduce el sobreajuste.

**Ejercicio 5:** Experimenta con diferentes funciones de activación (ReLU, ELU, LeakyReLU) en las capas lineales para ver cómo afectan al aprendizaje.

Pasos:
- Modifica las capas lineales en TextClassifier para utilizar diferentes funciones de activación.
- Entrena el modelo para cada configuración y evalúa el impacto en la rapidez de la convergencia y la precisión final.

In [ ]:
## Tus respuestas

Realizamos varias operaciones importantes para construir y entrenar un modelo de clasificación de texto utilizando la biblioteca fastai junto con PyTorch. 

In [ ]:
import torch
??torch.nn.RNN

Recuerda: Este fragmento de código utiliza la sintaxis de Python ?? para mostrar la documentación integrada de la clase torch.nn.RNN. Esto es útil para entender más detalles sobre cómo funciona la implementación de RNN en PyTorch, incluyendo parámetros y métodos disponibles.


In [ ]:
class TextClassifier(nn.Module):
    
    def __init__(self, vocab_size, hidden_size):
        super(TextClassifier, self).__init__()
        self.emb = nn.Embedding(vocab_size, hidden_size)
        self.rnn = nn.RNN(hidden_size, hidden_size, batch_first=True)
        self.fc1 = nn.Linear(hidden_size, 10)
        self.fc2 = nn.Linear(10, 2)
        
    def forward(self, x):
        
        x = self.emb(x)
        _, x = self.rnn(x)
        x = self.fc1(x)
        out = self.fc2(x)

        return out
     

learn = Learner(dls, TextClassifier(len(dls.vocab[0]), 100),
                loss_func=CrossEntropyLossFlat(), 
                metrics=accuracy)
learn.fit(10)
     


- nn.Embedding: Transforma índices de palabras en vectores densos de tamaño hidden_size.
- nn.RNN: Un módulo RNN que procesa secuencias de embeddings de palabras. El parámetro batch_first=True indica que la primera dimensión del tensor de entrada representa el tamaño del lote.
- nn.Linear: Dos capas lineales que transforman la salida de la RNN en logits para clasificación. La última capa tiene 2 unidades de salida, lo que sugiere una tarea de clasificación binaria.
- forward: Define cómo pasa la entrada a través del modelo. Nota que sólo se utiliza el último estado oculto de la RNN `(_, x = self.rnn(x))`  para la clasificación final, lo cual es común en tareas donde sólo el resultado después de ver toda la entrada es relevante (como en clasificación de sentimientos al final de la secuencia).

Luego se crea una nueva instancia de `Learner` y se entrena el modelo modificado por 10 épocas, Esto muestra cómo puedes iterar sobre la definición del modelo y el proceso de entrenamiento para experimentar con diferentes arquitecturas o configuraciones.

#### Redes bidireccionales

Las redes neuronales bidireccionales combinan dos RNNs que operan en direcciones opuestas sobre la entrada de datos, lo que permite que el modelo tenga en cuenta tanto el contexto pasado como el futuro en cada punto de la secuencia.

Al procesar una secuencia tanto en dirección hacia adelante como hacia atrás, una Bi-RNN puede capturar información de todo el contexto alrededor de un punto dado en la secuencia. Esto es especialmente útil para tareas donde la comprensión del contexto posterior es crucial, como puede ser en el etiquetado de partes del habla, reconocimiento de entidades nombradas, y otras tareas de procesamiento del lenguaje natural.

Dado que el modelo tiene acceso a más información contextual, a menudo puede hacer predicciones más precisas en comparación con una RNN unidireccional, particularmente en tareas complejas de clasificación o regresión sobre secuencias.

In [ ]:
class TextClassifier(nn.Module):
    
    def __init__(self, vocab_size, hidden_size):
        super(TextClassifier, self).__init__()
        self.emb = nn.Embedding(vocab_size, hidden_size)
        self.rnn = nn.RNN(hidden_size, hidden_size,
                          bidirectional=True, batch_first=True)
        self.fc1 = nn.Linear(hidden_size * 2, 10)
        self.fc2 = nn.Linear(10, 2)
        
    def forward(self, x):
        
        x = self.emb(x)
        _, x = self.rnn(x)
        x = torch.cat((x[0], x[1]), dim=-1)
        x = self.fc1(x)
        out = self.fc2(x)

        return out
     

learn = Learner(dls, TextClassifier(len(dls.vocab[0]), 100),
                loss_func=CrossEntropyLossFlat(), 
                metrics=accuracy)
learn.fit(10)
     

#### LSTM

In [ ]:
class TextClassifier(nn.Module):
    
    def __init__(self, vocab_size, hidden_size):
        super(TextClassifier, self).__init__()
        self.emb = nn.Embedding(vocab_size, hidden_size)
        self.rnn = nn.LSTM(hidden_size, hidden_size, batch_first=True)
        self.fc1 = nn.Linear(hidden_size, 10)
        self.fc2 = nn.Linear(10, 2)
        
    def forward(self, x):
        
        x = self.emb(x)
        x, _ = self.rnn(x)[1]
        x = self.fc1(x)
        out = self.fc2(x)
        
        return out
     

learn = Learner(dls, TextClassifier(len(dls.vocab[0]), 100),
                loss_func=CrossEntropyLossFlat(), 
                metrics=accuracy)
learn.fit(10)
     

#### GRU

In [ ]:


class TextClassifier(nn.Module):
    
    def __init__(self, vocab_size, hidden_size):
        super(TextClassifier, self).__init__()
        self.emb = nn.Embedding(vocab_size, hidden_size)
        self.rnn = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.fc1 = nn.Linear(hidden_size, 10)
        self.fc2 = nn.Linear(10, 2)
        
    def forward(self, x):
        
        x = self.emb(x)
        _, x = self.rnn(x)
        x = self.fc1(x)
        out = self.fc2(x)
        
        return out
     

learn = Learner(dls, TextClassifier(len(dls.vocab[0]), 100),
                loss_func=CrossEntropyLossFlat(), 
                metrics=accuracy)
learn.fit(10)
     
